In [ ]:
import os, random, pandas as pd, numpy as np
import pickle
import sys
import ast
import stanza
sys.path.append('../../../')
# sys.path.append('../ddi_preprocess')
from relation_extraction.data import utils
import nltk
from ast import literal_eval
import itertools
from ast import literal_eval # to convert the string tuple form to an actual tuple
RESOURCE_PATH = "/mnt/projects/OpenNRE/benchmark/raw_ddi/DDICorpus"
OUTPUT_PATH = "/mnt/projects/OpenNRE/benchmark/ddi"
outdir = 'original/'
def res(path): return os.path.join(RESOURCE_PATH, path)
def res(path): return os.path.join(OU, path)
from relation_extraction.data.converters.converter_ddi import get_dataset_dataframe, write_dataframe, \
read_dataframe, check_equality_of_written_and_read_df, write_into_txt, combine

In [ ]:
nlp = stanza.Pipeline('en', package='craft')

In [ ]:
df_train_drugbank = get_dataset_dataframe(nlp, res('Train/DrugBank/'), relation_extraction=True)

In [ ]:
df_train_medline = get_dataset_dataframe(res('Train/MedLine/'), relation_extraction=True)

In [ ]:
df_test_drugbank = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/DrugBank/'), relation_extraction=True)

In [ ]:
df_test_medline = get_dataset_dataframe(res('Test/test_for_ddi_extraction_task/MedLine/'), relation_extraction=True)

In [ ]:
len(df_train_drugbank)

In [ ]:
len(df_train_medline)

In [ ]:
len(df_test_drugbank)

In [ ]:
len(df_test_medline)

In [ ]:
def get_empty_entity_rows(df):
    empty_entity_rows = []
    def find_empty_entity_number(row):
        metadata = row.metadata
        e1 = metadata['e1']['word_index']
        e2 = metadata['e2']['word_index']
        if not e1 or not e2:
            empty_entity_rows.append(row.row_num)
    temp_df = df.copy()
    temp_df.insert(0, 'row_num', range(0, len(temp_df)))
    temp_df.apply(find_empty_entity_number, axis=1)
    return empty_entity_rows

def get_empty_rows_array(empty_entity_rows, df):
    empty_rows_array = []
    for index in empty_entity_rows:
        e1 = df.iloc[index].e1
        e2 = df.iloc[index].e2
        original_sentence = df.iloc[index].original_sentence
        tokenized_sentence = df.iloc[index].tokenized_sentence
        metadata = df.iloc[index].metadata
        empty_rows_array.append([index, original_sentence, e1, e2, metadata, tokenized_sentence])
    new_df = pd.DataFrame(data=empty_rows_array,    # values
             columns=['index_original', 'original_sentence' , 'e1', 'e2', 'metadata', 'tokenized_sentence'])
    return empty_rows_array, new_df

In [ ]:
def get_empty_vals(df):
    empty_entity_rows = get_empty_entity_rows(df)
    empty_rows_array, new_df = get_empty_rows_array(empty_entity_rows, df)
    return empty_rows_array, new_df

In [ ]:
get_empty_vals(df_train_drugbank)

In [ ]:
get_empty_vals(df_train_medline)

In [ ]:
get_empty_vals(df_test_drugbank)

In [ ]:
get_empty_vals(df_test_medline)

In [ ]:
if not os.path.exists(out(outdir)):
    os.makedirs(out(outdir))

In [ ]:
write_dataframe(df_train_drugbank, out(outdir + 'train_drugbank_original.csv'))

In [ ]:
df_train_drugbank_copy = read_dataframe(out(outdir + 'train_drugbank_original.csv'))

In [ ]:
# The first checks with the pd.equals method, and the other does a manual checking per column
check_equality_of_written_and_read_df(df_train_drugbank, df_train_drugbank_copy)

In [ ]:
write_dataframe(df_train_medline, out(outdir + 'train_medline_original.csv'))

In [ ]:
df_train_medline_copy = read_dataframe(out(outdir + 'train_medline_original.csv'))

In [ ]:
check_equality_of_written_and_read_df(df_train_medline, df_train_medline_copy)

In [ ]:
write_dataframe(df_test_drugbank, out(outdir + 'test_drugbank_original.csv'))

In [ ]:
df_test_drugbank_copy = read_dataframe(res(outdir + 'test_drugbank_original.csv'))

In [ ]:
check_equality_of_written_and_read_df(df_test_drugbank, df_test_drugbank_copy)

In [ ]:
write_dataframe(df_test_medline, out(outdir + 'test_medline_original.csv'))

In [ ]:
df_test_medline_copy = read_dataframe(out(outdir + 'test_medline_original.csv'))

In [ ]:
check_equality_of_written_and_read_df(df_test_medline, df_test_medline_copy)

In [ ]:
write_into_txt(df_train_drugbank, out(outdir + 'train_drugbank_original.txt'))

In [ ]:
write_into_txt(df_train_medline, out(outdir + 'train_medline_original.txt'))

In [ ]:
write_into_txt(df_test_drugbank, res(outdir + 'test_drugbank_original.txt'))

In [ ]:
write_into_txt(df_test_medline, res(outdir + 'test_medline_original.txt'))

In [ ]:
combine(out, outdir,  'train_drugbank_original', 'train_medline_original', 'ddi_train_original.txt')

In [ ]:
combine(res, outdir, 'test_drugbank_original', 'test_medline_original', 'ddi_val_original.txt')